<a href="https://colab.research.google.com/github/Balajilp/Deep_learning-ANN-_learning/blob/main/Decide_number_of_hidden_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [10]:
# importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
# loading the Dataset
df = pd.read_csv('/content/drive/MyDrive/DataSet/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
# splitting the dataframe into x and y co ordinates
x = df.iloc[:, 3:13]
y = df.iloc[:, 13]

In [13]:
# Create dummy variables
geography = pd.get_dummies(x['Geography'], drop_first=True)
gender = pd.get_dummies(x['Gender'], drop_first=True)

In [14]:
# concatenate the DataFrames
X = pd.concat([x, geography, gender], axis=1)

In [15]:
 # dropping the existing features
 X = X.drop(['Gender', 'Geography'], axis=1)

In [16]:
# Splitting the Dataset into training and test dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [31]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [32]:
# perform Hyperparameter optimization

# importing all the keras modules
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid

In [33]:
# creating a funcion for model creation
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
      if i == 0:
        model.add(Dense(nodes, input_dim=X_train.shape[1]))
        model.add(Activation(activation))
        model.add(Dropout(0.3))
      else:
        model.add(Dense(nodes))
        model.add(Activation(activation))
        model.add(Dropout(0.3))
    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
    

In [34]:
# initialize the function inside Keras Classifier
model = KerasClassifier(build_fn=create_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [35]:
# perform GridSearch CV Hyperparameter optimization
ayers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size=[128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)

In [36]:
# fitting the model inside gridsearchcv  runtime = 4.8 mins
grid_result = grid.fit(X_train, y_train)

In [38]:
# printing the best accuracy score and best parameters from the gridsearchcv
print(grid_result.best_score_, grid_result.best_params_)

0.8572499990463257 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [45, 30, 15]}


In [40]:
# predicting the x_test results output
pred_y = grid.predict(X_test)

In [41]:
y_pred = (pred_y > 0.5)

In [44]:
# Model evaluation
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred, y_test)
cm

array([[1537,  215],
       [  58,  190]])

In [45]:
# there is a slite increase in accuracy
score = accuracy_score(y_pred, y_test)
score

0.8635